In [41]:
pip install python-dotenv openai -q


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [42]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(), override=True)

True

In [43]:
import openai
import re
import httpx
import os

In [44]:
from openai import OpenAI

client = OpenAI()

MODEL = "gpt-4o-mini"
prompt = "Write something shot but funny"
chat_completion = client.chat.completions.create(
    model=MODEL, messages=[{"role": "user", "content": prompt}]
)

In [45]:
chat_completion.choices[0].message.content

'Why did the scarecrow win an award? Because he was outstanding in his field! 🌾😄'

# Creating the Agent Class

In [46]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []

        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, prompt):
        self.messages.append({"role": "user", "content": prompt})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self, model="gpt-4o-mini", temperature=0):
        completion = client.chat.completions.create(
            model=model, temperature=temperature, messages=self.messages
        )
        return completion.choices[0].message.content

# Creating the ReAct Prompt

In [47]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer

Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

get_cost:
e.g. get_cost: book
returns the cost of a book

wikipedia:
e.g. wikipedia: LangChain
Returns a summary from searching Wikipedia

Always look things up on Wikipedia if you have the opportunity to do so.

Example session #1:

Question: How much does a pen cost?
Thought: I should look the pen cost using get_cost
Action: get_cost: pen
PAUSE

You will be called again with this:

Observation: A pen costs $5

You then output:

Answer: A pen costs $5

Example session #2

Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:

Observation: France is a country. The capital is Paris.

You then output:

Answer: The capital of France is Paris
""".strip()


# The tools

In [48]:
def calculate(what):
    return eval(what)


def get_cost(thing):
    if thing in "pen":
        return "A pen costs $5"
    elif thing in "book":
        return "A book costs $20"
    elif thing in "stapler":
        return "A stapler costs $10"
    else:
        return "A random thing for writing costs $12"


def wikipedia(q):
    response = httpx.get(
        "https://en.wikipedia.org/w/api.php",
        params={"action": "query", "list": "search", "srsearch": q, "format": "json"},
        headers={"User-Agent": "ReAct/0.1 (contact@example.com)"},
    )
    results = response.json().get("query").get("search", [])

    if not results:
        return None
    return results[0]["snippet"]

In [49]:
wikipedia("langchain")

'Free and open-source software portal <span class="searchmatch">LangChain</span> is a software framework that helps facilitate the integration of large language models (LLMs) into applications'

In [50]:
known_actions = {"calculate": calculate, "get_cost": get_cost, "wikipedia": wikipedia}

# Testing the Agent

In [51]:
my_agent = Agent(prompt)

In [52]:
result = my_agent("How much does a pen cost?")
print(result)

Thought: I should look up the cost of a pen using the get_cost action.  
Action: get_cost: pen  
PAUSE


In [53]:
next_prompt = f"Observation: {get_cost('pen')}"
my_agent(next_prompt)

'Answer: A pen costs $5.'

In [54]:
my_agent.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\n\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_cost:\ne.g. get_cost: book\nreturns the cost of a book\n\nwikipedia:\ne.g. wikipedia: LangChain\nReturns a summary from searching Wikipedia\n\nAlways look things up on Wikipedia if you have the opportunity to do so.\n\nExample session #1:\n\nQuestion: How much does a pen cost?\nThought: I should look the pen cost using get_cost\nAction: get_cost: pen\nPAUSE\n\nYou will be called again with this:\n\nObservation: A pen costs $5\n\nYou then output:\n\nAnswer

In [56]:
abot = Agent(prompt)

In [57]:
question = """I want to buy a pen and a book.
How much do they cost in total?"""

abot(question)

'Thought: I need to find the cost of both a pen and a book to calculate the total cost. I will first get the cost of each item using the get_cost action. \nAction: get_cost: pen\nPAUSE'

In [58]:
next_prompt = f"Observation: {get_cost('pen')}"
print(next_prompt)

Observation: A pen costs $5


In [59]:
abot(next_prompt)

'Thought: Now that I have the cost of the pen, I need to find the cost of the book to calculate the total. \nAction: get_cost: book\nPAUSE'

In [ ]:
next_prompt = f"Observation: {get_cost('book')}"
print(next_prompt)

Observation: A book costs $20


In [61]:
abot(next_prompt)

'Thought: I now have the costs of both the pen and the book. The pen costs $5 and the book costs $20. I will calculate the total cost by adding these two amounts together. \nAction: calculate: 5 + 20\nPAUSE'

In [62]:
abot = Agent(prompt)

In [63]:
query = "2024 United States elections"
abot(query)

'Thought: I should look up information about the 2024 United States elections on Wikipedia to get a comprehensive overview.  \nAction: wikipedia: 2024 United States elections  \nPAUSE'

In [64]:
next_prompt = f"Observation: {wikipedia(query)}"
print(next_prompt)


Observation: <span class="searchmatch">Elections</span> were held in the <span class="searchmatch">United</span> <span class="searchmatch">States</span> on November 5, <span class="searchmatch">2024</span>. In the presidential <span class="searchmatch">election</span>, former Republican President Donald Trump, seeking a non-consecutive


In [65]:
abot(next_prompt)

'Answer: Elections will be held in the United States on November 5, 2024, including a presidential election where former Republican President Donald Trump is seeking a non-consecutive term.'

# Automating the Agent

In [68]:
action_re = re.compile(r"^Action: (\w+): (.*)$")

In [69]:
def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question

    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)

        actions = [action_re.match(a) for a in result.split("\n") if action_re.match(a)]

        if actions:
            action, action_input = actions[0].groups()

            if action not in known_actions:
                raise Exception(f"Unknown action: {action}: {action_input}")

            print(f"--running {action} {action_input}")
            observation = known_actions[action](action_input)
            print(f"Observation:{observation}")
            next_prompt = f"Observation: {observation}"
        else:
            return


In [70]:
question = """I want to buy 2 books and 3 pens. How much do i have to pay?"""
query(question)

Thought: I need to find the cost of a book and a pen to calculate the total cost for 2 books and 3 pens. I will first look up the cost of a book and then the cost of a pen. 

Action: get_cost: book
PAUSE
--running get_cost book
Observation:A book costs $20
Thought: Now that I know the cost of a book is $20, I need to find the cost of a pen to complete the calculation for the total cost of 2 books and 3 pens.

Action: get_cost: pen
PAUSE
--running get_cost pen
Observation:A pen costs $5
Thought: I now have the costs: a book costs $20 and a pen costs $5. I can calculate the total cost for 2 books and 3 pens using these prices.

Action: calculate: 2 * 20 + 3 * 5
PAUSE
--running calculate 2 * 20 + 3 * 5
Observation:55
Answer: You have to pay $55 for 2 books and 3 pens.


In [72]:
question = """EUFA Euro 2021"""
query(question)

Thought: I should look up information about the EUFA Euro 2021 on Wikipedia to get a summary of the event.  
Action: wikipedia: UEFA Euro 2021  
PAUSE
--running wikipedia UEFA Euro 2021  
Observation:The 2020 <span class="searchmatch">UEFA</span> European Football Championship, commonly referred to as <span class="searchmatch">UEFA</span> <span class="searchmatch">Euro</span> 2020 or simply <span class="searchmatch">Euro</span> 2020, was the 16th <span class="searchmatch">UEFA</span> European Championship, the
Answer: The UEFA Euro 2020 was the 16th UEFA European Championship.
